# Segmenting and Clustering Neighborhoods in Toronto

## Syed Khaleelullah - Applied Data Science Capstone - Week3 Final Assignment - Part 3

#### In this Assignment, we will explore, segment and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.  What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project. 

#### For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will perform to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format. Once this is done, we will explore and cluster the neighborhoods in the city of Toronto.

Tips for performing WebScraping is found using this link: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/NewLinkWebscrapingHints.md

## QUESTION 1

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [5]:
# create a new DataFrame from the three lists
toronto_df=pd.DataFrame(table_contents)

toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto',
                                             'EtobicokeNorthwest':'Etobicoke','East YorkEast Toronto':'East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
#print the table contents
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
# print the number of rows of the cleaned dataframe
toronto_df.shape

(103, 3)

## QUESTION 2

In [8]:
pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


Alternatively, as suggested in the assignment, I will be using the provided CSV file of the geographical coordinates of each postal code

In [9]:
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
print("The shape of our wikipedia Toronto data is: ", toronto_df.shape)
print("the shape of our csv Toronto data is: ", data.shape)

The shape of our wikipedia Toronto data is:  (103, 3)
the shape of our csv Toronto data is:  (103, 3)


Here, we can see that the shape of both the data is same.
Hence, we can perform to join the data.

In [12]:
toronto_combined_data = toronto_df.merge(data, on='PostalCode', how="left")
toronto_combined_data.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [13]:
toronto_combined_data.shape

(103, 5)

## QUESTION 3

In [14]:
from geopy.geocoders import Nominatim

In [15]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_combined_data['Borough'].unique()),
        toronto_combined_data.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [17]:
toronto_combined_data['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

### Let's create a Map of Toronto

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_combined_data['Latitude'], toronto_combined_data['Longitude'], toronto_combined_data['Borough'], toronto_combined_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Filter only the Borough that contain the word Scarborough into a DataFrame

In [19]:
scarborough_data = toronto_combined_data[toronto_combined_data['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [20]:
scarborough_data.shape

(17, 5)

In [21]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [22]:
# create map of Toronto using latitude and longitude values
map_scar = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Borough'], scarborough_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scar)  
    
map_scar

### Lets use the Foursquare API to explore the neighborhoods

In [42]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Now, let's get the top 100 venues that are within a radius of 500 meters.

In [24]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['PostalCode'], scarborough_data['Borough'], scarborough_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [25]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(91, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


#### Here, we can check how many venues we got for each Neighborhood

In [26]:
venues_df.groupby(["Neighborhood"]).count()

,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
Agincourt,4,4,4,4,4,4,4,4
"Birch Cliff, Cliffside West",5,5,5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",13,13,13,13,13,13,13,13
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9,9,9
"Guildwood, Morningside, West Hill",9,9,9,9,9,9,9,9
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5,5,5


#### Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 57 uniques categories.


In [28]:
venues_df['VenueCategory'].unique()[:50]

array(['Fast Food Restaurant', 'Bar', 'Bank', 'Electronics Store',
       'Restaurant', 'Mexican Restaurant', 'Rental Car Location',
       'Donut Shop', 'Medical Center', 'Intersection', 'Breakfast Spot',
       'Coffee Shop', 'Korean BBQ Restaurant', 'Soccer Field',
       'Caribbean Restaurant', 'Hakka Restaurant', 'Thai Restaurant',
       'Athletics & Sports', 'Gas Station', 'Bakery',
       'Fried Chicken Joint', 'Playground', 'Department Store',
       'Discount Store', 'Chinese Restaurant', 'Bus Station',
       'Ice Cream Shop', 'Bus Line', 'Metro Station', 'Park', 'Motel',
       'American Restaurant', 'Café', 'General Entertainment', 'Farm',
       'Skating Rink', 'College Stadium', 'Indian Restaurant',
       'Pet Store', 'Vietnamese Restaurant', 'Brewery', 'Gaming Cafe',
       'Sandwich Place', 'Middle Eastern Restaurant', 'Shopping Mall',
       'Smoke Shop', 'Auto Garage', 'Latin American Restaurant', 'Lounge',
       'Clothing Store'], dtype=object)

### Let's Analyze Each Area

In [29]:
#################################################################
# one hot encoding
scarb_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['PostalCode'] = venues_df['PostalCode']
scarb_onehot['Borough'] = venues_df['Borough'] 
scarb_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(scarb_onehot.columns[-3:]) + list(scarb_onehot.columns[:-3])
scarb_onehot = scarb_onehot[fixed_columns]

# # move neighborhood column to the first column
# fixed_columns = [scarb_onehot.columns[-3:]] + list(scarb_onehot.columns[:-3])
# scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,PostalCode,Borough,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Donut Shop,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,General Entertainment,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [30]:
scarb_onehot.shape

(91, 60)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
scarb_grouped = scarb_onehot.groupby(["Neighborhood"]).mean().reset_index()
scarb_grouped.head(10)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Donut Shop,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,General Entertainment,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.25,0.25,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.2,0.000,0.000000,0.00,0.0,0.2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.2,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.2,0.0,0.000000,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.0,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.125,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.125000,0.0,0.000000,0.125000,0.0,0.125,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.125000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.0,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.076923,0.00,0.0,0.0,0.076923,0.0,0.0,0.0,0.000000,0.000000,0.0,0.076923,0.076923,0.0,0.000000,0.076923,0.0,0.000,0.000000,0.000000,0.076923,0.076923,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.076923,0.000000,0.000000,0.076923,0.153846,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.076923,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,0.000000,0.0,0.000,0.142857,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.142857,0.000000,0.0,0.000,0.000000,0.285714,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.0,0.222222,0.000000,0.0,0.000000,0.000000,0.111111,0.111111,0.0,0.000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.111111,0.000000,0.111111,0.000000,0.0,0.00,0.00,0.000000,0.111111,0.000000,0.0,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.111111,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000,0.0,0.000000,0.111111,0.0,0.111111,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.111111,0.111111

#### Now let's create the new dataframe and display the top 10 venues for each Neighbourhood.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Breakfast Spot,Latin American Restaurant,Lounge,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Farm,Café,College Stadium,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Gas Station,Bank,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Bakery,Thai Restaurant,Athletics & Sports,Furniture / Home Store,Convenience Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant,Intersection,Italian Restaurant,Convenience Store,Fried Chicken Joint,Noodle House,Pharmacy
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop,Discount Store,Department Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Brewery,Gaming Cafe,Pet Store,Chinese Restaurant,Bar,Convenience Store,Fried Chicken Joint,Fast Food Restaurant
6,"Golden Mile, Clairlea, Oakridge",Bakery,Intersection,Soccer Field,Ice Cream Shop,Park,Metro Station,Bus Line,Bus Station,Discount Store,Convenience Store
7,"Guildwood, Morningside, West Hill",Breakfast Spot,Donut Shop,Medical Center,Intersection,Rental Car Location,Restaurant,Mexican Restaurant,Bank,Electronics Store,Coffee Shop
8,"Kennedy Park, Ionview, East Birchmount Park",Coffee Shop,Discount Store,Department Store,Bus Station,Chinese Restaurant,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Farm,Electronics Store,Donut Shop,Discount Store,Department Store


### Let's Cluster the Areas

In [34]:
########################################################################
scarb_data = scarborough_data.drop(16)

# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 3, 0, 0, 0, 0, 1], dtype=int32)

### Add kmeans.labels_ here

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
scarb_merged = scarb_data

# add clustering labels
scarb_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")

print(scarb_merged.shape)
scarb_merged.head() # check the last columns!

(16, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Farm,Electronics Store,Donut Shop,Discount Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Breakfast Spot,Donut Shop,Medical Center,Intersection,Rental Car Location,Restaurant,Mexican Restaurant,Bank,Electronics Store,Coffee Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Vietnamese Restaurant,Clothing Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Gas Station,Bank,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Bakery,Thai Restaurant,Athletics & Sports,Furniture / Home Store,Convenience Store


### Let's Visualize the Clusters now

In [36]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's examine each of the Five Clusters

#### Cluster 1

In [37]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + [2] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",0,Fast Food Restaurant,Vietnamese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Farm,Electronics Store,Donut Shop,Discount Store,Department Store
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",0,Bar,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop,Discount Store
2,Scarborough,"Guildwood, Morningside, West Hill",0,Breakfast Spot,Donut Shop,Medical Center,Intersection,Rental Car Location,Restaurant,Mexican Restaurant,Bank,Electronics Store,Coffee Shop
3,Scarborough,Woburn,0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Vietnamese Restaurant,Clothing Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop
5,Scarborough,Scarborough Village,0,Playground,Vietnamese Restaurant,Clothing Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop,Discount Store
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",0,Coffee Shop,Discount Store,Department Store,Bus Station,Chinese Restaurant,Vietnamese Restaurant,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
7,Scarborough,"Golden Mile, Clairlea, Oakridge",0,Bakery,Intersection,Soccer Field,Ice Cream Shop,Park,Metro Station,Bus Line,Bus Station,Discount Store,Convenience Store
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",0,American Restaurant,Motel,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop,Discount Store,Department Store
10,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",0,Indian Restaurant,Vietnamese Restaurant,Brewery,Gaming Cafe,Pet Store,Chinese Restaurant,Bar,Convenience Store,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",0,Pizza Place,Chinese Restaurant,Gas Station,Thai Restaurant,Intersection,Italian Restaurant,Convenience Store,Fried Chicken Joint,Noodle House,Pharmacy


In [38]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + [2] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,"Birch Cliff, Cliffside West",1,General Entertainment,Skating Rink,Farm,Café,College Stadium,Coffee Shop,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [39]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + [2] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,Agincourt,2,Clothing Store,Breakfast Spot,Latin American Restaurant,Lounge,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Donut Shop


In [40]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + [2] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,Cedarbrae,3,Gas Station,Bank,Caribbean Restaurant,Hakka Restaurant,Fried Chicken Joint,Bakery,Thai Restaurant,Athletics & Sports,Furniture / Home Store,Convenience Store


In [41]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + [2] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,"Wexford, Maryvale",4,Middle Eastern Restaurant,Smoke Shop,Auto Garage,Shopping Mall,Sandwich Place,Bakery,Vietnamese Restaurant,Department Store,College Stadium,Convenience Store
